In [1]:
import requests
from pathlib import Path
import xmltodict
import os
import pandas as pd
import io
import datetime
import pymongo
import json
import gzip
import dns
from os import listdir
from os.path import isfile, join

urlBase='https://tisvcloud.freeway.gov.tw/history/cms/'  # 20190624/cms_value_0000.xml.gz   20190624/cms_value_0014.xml.gz
baseDir='../../../data/'
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.traffic
trType='cms'
collect = db[trType]

#確認該 url 是否可以下載, 例如 20190230 就不存在, 無法下載; 而且必須是附件形式的方可
def isDownloadable(url):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if content_type is None:
        return False
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return True


#下載某url檔案後, 放在指定目錄下
def downloadFileFromUrl(url, directory):
    filename = directory+'/'+ url.rsplit('/', 1)[1]
    if not os.path.exists(filename):
        r = requests.get(url, allow_redirects=True)
        open(filename, 'wb').write(r.content)

In [2]:
#抓取 CMS 的某一天所有可能的檔案 並下載到指定目錄
def downloadCMSDay(trType, day, baseDir):
    downloads=[]
    for hour in range(0,24):
        for mininute in range(0,60,2):  #cms 每2分鐘一次
            downloads.append(urlBase + day + '/' + trType + '_value_'+ format(hour, '02d')+ format(mininute, '02d') +'.xml.gz' )
    p=baseDir+trType+'/'+day
    Path(p).mkdir(parents=True, exist_ok=True)
    for url in downloads:
        if isDownloadable(url):
            downloadFileFromUrl(url,p)
            #print(url)  

In [3]:
#days=['0404','0405','0406','0407','0606','0607','0608','0624','0625','0626','0627','0628'] #
days=['0407','0606','0202','0203','0204','0205','0206','0207','0208','0209','0210'] #
for d in days:
    downloadCMSDay('cms','2019'+d,baseDir)
    